In [12]:
!pip install pandas langchain faiss-cpu sentence-transformers openai

In [13]:
import os

# List all files in the /content directory
for f in os.listdir("/content"):
    print(f)


.config
Training Dataset.csv
sample_data


In [14]:
import pandas as pd

df = pd.read_csv("/content/Training Dataset.csv")  # Make sure file is still uploaded
df.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [15]:
# Convert each row to a string for embedding
texts = []

for idx, row in df.iterrows():
    text = "\n".join([f"{col}: {row[col]}" for col in df.columns])
    texts.append(text)

# Show a sample
print(texts[0])


Loan_ID: LP001002
Gender: Male
Married: No
Dependents: 0
Education: Graduate
Self_Employed: No
ApplicantIncome: 5849
CoapplicantIncome: 0.0
LoanAmount: nan
Loan_Amount_Term: 360.0
Credit_History: 1.0
Property_Area: Urban
Loan_Status: Y


In [16]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load a lightweight embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
embeddings = model.encode(texts)

# Create FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

print(f"✅ Stored {len(embeddings)} vectors in FAISS index.")


✅ Stored 614 vectors in FAISS index.


In [18]:
from transformers import pipeline

# ✅ Load a lightweight open-source model (T5-style)
qa_model = pipeline("text2text-generation", model="google/flan-t5-base")

# 🔁 Replace ask_gpt with this offline version
def ask_local(question):
    context = "\n\n".join(get_top_k_matches(question))
    prompt = f"Context: {context}\n\nQuestion: {question}\nAnswer:"
    response = qa_model(prompt, max_new_tokens=100)
    return response[0]['generated_text']

# 🔍 Try a question
question = "What kind of applicants usually get loan approval?"
print("🤖 Answer:\n", ask_local(question))


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


🤖 Answer:
 male and female


In [19]:
print(ask_local("What factors affect loan approval?"))
print(ask_local("How important is credit history for getting a loan?"))
print(ask_local("Do married applicants get more approvals?"))
print(ask_local("What type of education is most common among approved applicants?"))
print(ask_local("Does being self-employed reduce chances of approval?"))


Gender, Married, Dependents, 1 Education, 0 CoapplicantIncome, 1083.0 Loan Amount: 135.0 Loan Amount_Term: 360.0 Credit History: 1.0 Property Area: Urban Loan_Status: N Loan_ID: LP001259 Gender, Male, Married, Yes Dependents, 1 Education, 0 CoapplicantIncome: 1083.0 Loan Amount
1.0
No
Graduate
No
